In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets

# Importando nuestros datos

Comenzamos obteniendo los archivos que contienen nuestros datasets, definiendo un path para cada uno para despues importarlos como dataframes

In [2]:
data_path_sniim = os.path.join("..","..","data","clean_data","precios_sniim.parquet")
data_path_siap = os.path.join("..","..","data","clean_data","siap.parquet.zip")
data_path_siap_wide = os.path.join("..","..","data","clean_data","siap_wide.parquet.zip")

In [3]:
df_precios_sniim = pd.read_parquet(data_path_sniim)
df_cultivos_siap = pd.read_parquet(data_path_siap)
df_cultivos_siap_wide = pd.read_parquet(data_path_siap_wide)

Después, podemos obtener algunas muestras de nuestras dataframes para verificar su estructura, las cuales están especificadas en nuestros diccionario de datos.

In [23]:
df_precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,año,mes
453946,2020-10-23,Ciruela Roja,Caja de 10 kg.,Importación,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",50.00,55.00,50.00,2020,10
863381,2022-10-14,Guayaba,Caja de 10 kg.,Michoacán,Coahuila,"Central de Abasto de La Laguna, Torreón",20.00,21.30,21.20,2022,10
1021811,2021-08-26,Platano Tabasco,Caja de 18 kg.,Chiapas,Chihuahua,Mercado de Abasto de Cd. Juárez,12.78,12.78,12.78,2021,8
1358919,2023-05-26,Tomate Saladette,Kilogramo,Jalisco,Chihuahua,Mercado de Abasto de Cd. Juárez,22.50,22.50,22.50,2023,5
1069180,2021-04-19,Lima,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,22.00,24.00,23.00,2021,4
905127,2021-04-14,Naranja Valencia Grande,Kilogramo,Veracruz,Michoacán,Mercado de Abasto de Morelia,9.00,10.00,10.00,2021,4
870030,2022-08-25,Guayaba,Caja de 10 kg.,Veracruz,Veracruz,Central de Abasto de Minatitlán,21.00,24.00,22.00,2022,8
758960,2022-07-05,Cilantro,Kilogramo,Jalisco,Jalisco,Mercado de Abasto de Guadalajara,16.75,17.00,16.75,2022,7
692309,2020-10-01,Guayaba,Caja de 16 kg.,Michoacán,Michoacán,Mercado de Abasto de Morelia,16.88,19.69,19.69,2020,10
269307,2023-09-22,Brocoli,Caja de 8 kg.,Puebla,Sonora,"Mercado de Abasto ""Francisco I. Madero"" de Her...",37.50,40.63,40.63,2023,9


In [24]:
df_cultivos_siap_wide.sample(10)

,Entidad,Municipio,Sembrada (ha),Cosechada (ha),Siniestrada (ha),Producción,Rendimiento (udm/ha),Cultivo,fecha
91,Ciudad de México,Tláhuac,1.8,1.8,0.0,43.20,24.00,alfalfa,2022-01-01
149,Chihuahua,Coronado,35.0,22.0,0.0,22.00,1.00,frijol,2021-10-01
52,Tamaulipas,Soto La Marina,3.0,0.0,0.0,0.00,0.00,toronja (pomelo),2020-07-01
299,Oaxaca,San Pablo Villa de Mitla,116.5,7.5,0.0,485.40,64.72,agave,2023-08-01
94,Puebla,San Jerónimo Tecuanipan,10.0,0.0,0.0,0.00,0.00,brócoli,2021-08-01
129,Tamaulipas,Bustamante,140.5,80.0,0.0,92.80,1.16,tuna,2020-09-01
9,Jalisco,La Manzanilla de la Paz,2.0,2.0,0.0,4725.00,2362.50,rosa,2023-09-01
159,Quintana Roo,Bacalar,35.0,35.0,0.0,623.70,17.82,papaya,2020-06-01
226,Oaxaca,San Juan Ozolotepec,281.0,278.0,0.0,336.38,1.21,café cereza,2022-09-01
115,Guanajuato,León,111.5,111.5,0.0,2407.50,21.59,chile verde,2021-10-01


Antes de proseguir, podemos modificar las propiedades de seaborn para las visualizaciones que haremos mas adelante, seleccionando el estilo `whitegrid` y ajustando el tamaño de fuente a 1.2:

In [6]:
sns.set_style("whitegrid")
sns.set(font_scale=1.2)

# Analisis para datos sobre precios SNIIM

Antes de comenzar, crearemos un par de columnas en nuestra dataframe para indicar tanto el mes como el año de cada entrada:

In [7]:
df_precios_sniim["año"] = df_precios_sniim["fecha"].dt.year
df_precios_sniim["mes"] = df_precios_sniim["fecha"].dt.month

## Tendencia historica de los precios (2020 - 2023)

In [8]:
def graficar_precios(cultivo):
    subset_df = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
    
    subset_df = subset_df.resample('D', on='fecha').mean(numeric_only=True)
    subset_df["ewm"] = subset_df["precio_frec"].ewm(span=14).mean()
    #subset_df["año"] = subset_df.index.year
    
    fig,ax = plt.subplots()
    
    sns.lineplot(x="fecha",
                 y="ewm",
                 hue="año",
                 data=subset_df,
                 errorbar=None,
                 ax=ax,
                 palette="tab10")
    
    sns.scatterplot(x="fecha",
                    y="precio_frec",
                    hue="año",
                    data=subset_df,
                    ax=ax,
                    palette="tab10",
                    s=25,
                    alpha=0.5)
    
    ax.title.set_text(f"Historial de precios para {cultivo} (promedio diario)")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Precio por kg, MXN")
    ax.grid(axis='y')
    ax.tick_params(axis='x',rotation=90);
    ax.legend(title='Año', labels=["2020","2021","2022","2023"])
    
    fig.set_size_inches(16,9)
    
    plt.show()
    
    return

In [9]:
lista_productos = sorted(list(df_precios_sniim["producto"].unique()))
lista_destinos = sorted(list(df_precios_sniim["destino"].unique()))
lista_anios = [2020,2021,2022,2023]
#lista_productos[:5]

In [10]:
dd_productos = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

widgets.interact(graficar_precios,cultivo=dd_productos)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.graficar_precios(cultivo)>

In [11]:
def grafica_anio(cultivo,anio):
    subset_df = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
    subset_df = subset_df[subset_df["año"] == anio]
    
    try:
        subset_df = subset_df.resample('D', on='fecha').mean(numeric_only=True)
        subset_df["ewm"] = subset_df["precio_frec"].ewm(span=7).mean()

        fig,ax = plt.subplots()

        sns.lineplot(x="fecha",
                     y="ewm",
                     data=subset_df,
                     errorbar=None,
                     ax=ax)

        sns.scatterplot(x="fecha",
                        y="precio_frec",
                        data=subset_df,
                        ax=ax,
                        s=25,
                        alpha=0.5)

        ax.title.set_text(f"Historial de precios para {cultivo}, año: {anio} (promedio diario)")
        ax.set_xlabel("Fecha")
        ax.set_ylabel("Precio por kg, MXN")
        ax.grid(axis='y')
        ax.tick_params(axis='x',rotation=90);

        fig.set_size_inches(16,9)
        
        subset_df2 = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
        subset_df2 = subset_df2[subset_df2["año"] == anio]
        
        fecha_max = subset_df2[subset_df2["precio_max"] == subset_df2["precio_max"].max()]["fecha"].iloc[0]
        central_max = subset_df2[subset_df2["precio_max"] == subset_df2["precio_max"].max()]["central"].iloc[0]
        precio_max = subset_df2["precio_max"].max()
        
        print(f"Precio maximo: ${precio_max:.2f}/kg el {fecha_max.strftime('%d-%m-%Y')} en {central_max}")
        
        fecha_min = subset_df2[subset_df2["precio_min"] == subset_df2["precio_min"].min()]["fecha"].iloc[0]
        central_min = subset_df2[subset_df2["precio_min"] == subset_df2["precio_min"].min()]["central"].iloc[0]
        precio_min = subset_df2["precio_min"].min()
        
        print(f"Precio minimo: ${precio_min:.2f}/kg el {fecha_min.strftime('%d-%m-%Y')} en {central_min}")

        plt.show()
    
    except:
        print("No hay datos para esa combinacion")
    
    return

In [12]:
dd_productos = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

dd_anios = widgets.Dropdown(options=lista_anios,
                      value=lista_anios[-1],
                      description="Año: ")

widgets.interact(grafica_anio,cultivo=dd_productos,anio=dd_anios)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.grafica_anio(cultivo, anio)>

In [21]:
def graficar_dist_precios(producto):
    df = df_precios_sniim.copy()
    
    df_filter = df[(df["producto"] == producto) & (df["año"] == 2023)]
    
    try:
        fig, (ax1,ax2) = plt.subplots(2,1)

        sns.boxplot(x="mes",
                    y="precio_frec",
                    data=df_filter,
                    ax=ax1)
        

        ax1.title.set_text(f"Diagrama de cajas de precios por mes para {producto}")
        ax1.set_xlabel("Mes")
        ax1.set_ylabel("Precio por kg, MXN")
        
        sns.histplot(x="precio_frec",
                   data=df_filter,
                   ax=ax2,kde=True)
        
        ax2.title.set_text(f"Histograma de precios en el año para {producto}")
        ax2.set_xlabel("Precio mas frecuente")

        fig.set_size_inches(16,16)
        plt.tight_layout()
        plt.show()
    
    except:
        raise ValueError("Error! Parece que algo salio mal.")

In [22]:
dd_productos_hist = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

widgets.interact(graficar_dist_precios,producto = dd_productos_hist)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.graficar_dist_precios(producto)>

In [15]:
def graficar_promedios_mensuales(producto,central):
    df = df_precios_sniim.copy()
    
    df_filter = df[(df["producto"] == producto) & (df["año"] == 2023)]
    df_grouped = df_filter.groupby(["central","mes"])["precio_frec"].mean().to_frame()
    
    df_unstacked = df_grouped.unstack()
    df_unstacked.columns = df_unstacked.columns.droplevel()
    df_unstacked = df_unstacked.reset_index()
    df_unstacked.columns.name=None
    
    df_melted = df_unstacked.melt(id_vars=["central"],var_name="mes",value_name="precio_promedio").dropna()
    
    dict_meses = {1: "Enero",
                  2: "Febrero",
                  3: "Marzo",
                  4: "Abril",
                  5: "Mayo",
                  6: "Junio",
                  7: "Julio",
                  8: "Agosto",
                  9: "Septiembre",
                  10: "Octubre",
                  11: "Noviembre",
                  12: "Diciembre"}
    
    df_melted["mes"] = df_melted["mes"].map(dict_meses)
    df_final = df_melted[df_melted["central"] == central]
    
    try:
        fig, ax = plt.subplots()

        sns.pointplot(x="mes",
                      y="precio_promedio",
                      data=df_final,
                      ax=ax,
                      color='r',
                      errorbar=None)

        ax.title.set_text(f"Precios mensuales para {producto} en {central} (promedio mensual)")
        ax.set_xlabel("Mes")
        ax.set_ylabel("Precio por kg, MXN")
        ax.grid(axis='x')

        fig.set_size_inches(16,9)
        plt.show()
    
    except:
        raise ValueError(f"No existen datos para {producto} en {central}")

In [16]:
def obtener_valores_unicos(df,columna):
    return sorted(list(df[columna].unique()))

In [17]:
def generar_grafica_precios_meses(producto):
    df = df_precios_sniim.copy()
    subset_df = df[(df["producto"] == producto) & (df["año"] == 2023)]
    
    lista_centrales = obtener_valores_unicos(subset_df,"central")
    
    dd_centrales = widgets.Dropdown(options=lista_centrales,
                      value=lista_centrales[0],
                      description="Central: ")
    
    widgets.interact(graficar_promedios_mensuales,producto=widgets.fixed(producto),central=dd_centrales)

In [18]:
dd_productos2 = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

widgets.interact(generar_grafica_precios_meses,producto=dd_productos2)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.generar_grafica_precios_meses(producto)>

## Utilizando categorias del SIAP